Predicción de la evolución de la diabetes en pacientes usando MLP
===

Se desea construir um modelo de regresión no lineal (redes neuronales artificiales) que permita pronósticar el progreso de la diabetes con un horizonte de doce meses con base en variables físicas y pruebas de laboratorio. Véase https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

En este problema se tiene una base de datos de diez variables base (edad, sexo, índice de masa corporal, presión arterial, y seis variables medidas en sangre) para 442 pacientes, y un índice que mide el progreso de la diabetes un año después de la prueba. La columna Y es la variable explicada.

In [1]:
#
# La muestra se encuentra dividida en tres partes:
#
#   * X_train, y_true_train: es la muestra para estimar los parametros optimos
#
#   * X_test, y_true_test: es la muestra para seleccionar la mejor configuración
#
#   * X_val, y_true_val: es la muestra para probar el modelo en productivo
#
import warnings

import pandas as pd
import pytest
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")

df = pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/diabetes.csv")

In [2]:
#
# Use la muestra (X_train, y_true_train) para la estimación
# de los pesos óptimos de la red neuronal.
#
# Seleccione el modelo óptimo como aquel que minimice el error
# cuadrático medio para la muestra (X_test, y_true_test).
#
# Considere únicamente modelos desde una (1) hasta (5) 
# neuronas en la capa oculta. Considere únicamente las
# siguientes semillas para inicializar la red neuronal
# 1000, 1001, 1002, 1003, 1004, 1005.
#
# Compute el error cuadrático medio para la muestra
# (X_val, y_true_val). Esta muestra representa la operación
# del modelo en productio
# 
# Rta/
# True
#

# >>> Inserte su codigo aquí >>>
# !pip install fast_ml
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from fast_ml.model_development import train_valid_test_split

new_df = MinMaxScaler(feature_range=(0, 0.5)).fit_transform(df)
columns = list(df.columns)
data = pd.DataFrame(new_df, columns=columns)

X = data.drop(['target'],1)
y = data['target']

X_train, X_tem, y_true_train, y_tem = train_test_split(X, y, test_size=0.3, random_state=603)
X_test, X_val, y_true_test, y_true_val = train_test_split(X_tem,y_tem, test_size=0.5, random_state=603)

# X_train, y_true_train, X_val, y_true_val, X_test, y_true_test = train_valid_test_split(data, target = 'target',
#                                                                       train_size=0.7, valid_size=0.15, test_size=0.15, random_state=177)

param_grid = [
{
  "hidden_layer_sizes": [(1,),(2,),(3,),(4,),(5,)],
  "random_state":[1000,1001,1002,1003,1004,1005],
  "activation": ['identity'],
  "solver": ['adam'],
  "learning_rate": ['constant', 'invscaling', 'adaptive']
}
]

m = MLPRegressor(max_iter=1000)
gs_nn = GridSearchCV(estimator=m, param_grid=param_grid, cv=5, n_jobs=-1).fit(X_train, y_true_train)

y_pred = gs_nn.predict(X_test)
mse = mean_squared_error(y_true_test, y_pred)
x_pred = gs_nn.predict(X_val)
mse_val = mean_squared_error(y_true_val, x_pred)
# <<<

# ---->>> Evaluación ---->>>
pytest.approx(mse_val, 0.0001) == 0.009535

True